# Objective Finetuning Quickstart

Improve your Indexes by integrating your own data with Finetuning. Objective Finetuning amplifies the capabilities of AI search by incorporating your unique insights — your data and your feedback.

Read the full [Quickstart Doc](https://www.objective.inc/docs/quickstarts/finetuning-quickstart).


## Setup

Begin by obtaining an API key and installing the SDK:

To get an API key, [join the waitlist](https://objective.inc/contact).

```bash
pip install objective-sdk
```

## Adding Data

Begin by adding Objects to the Object Store. The code below will download 10,000 example e-commerce products and upsert them to the Object Store. In this example we are building an e-commerce search platform:

In [ ]:
import json
import requests
from objective import Client

client = Client(api_key="YOUR_API_KEY")

data = json.loads(requests.get("https://d11p8vtjlacpl4.cloudfront.net/demos/ecommerce/hm-10k.json").text)

client.object_store.upsert_objects([{"id": str(obj.get("article_id")), "object": obj} for obj in data])

## Building an Index

Build a `text` index out of the Objects pushed to the API with `searchable` and `filterable` fields:

In [ ]:
index = client.indexes.create_index(
    index_type="text", # Specify the type of index as 'text'
    fields={
      # Defines fields that will be indexed, their values will influence relevance
      "searchable": ["prod_name", "department_name", "detail_desc", "garment_group_name", "index_group_name", "index_name", "perceived_colour_master_name", "perceived_colour_value_name"],
      # Defines fields that will be filterable
      "filterable": ["department_name", "colour_group_name", "garment_group_name", "index_group_name", "index_name"]
    }
)
index.status(watch=True)

## Create Finetuned Index

Finetuning requires feedback data. Namely, examples of queries with relevance judgements on their results. The data takes the form of `(query, object_id, label)`, where `object_id` is the ID of the search result in the Object Store, and `label` is the rating on a `{BAD, OK, GREAT}` scale. 

Here are example grades of a search for dark blue polo shirt:

In [12]:
with open("graded_input.jsonl", "r") as file:
    for line in file:
        data = json.loads(line)
        if data["query"] == "dark blue polo shirt":
            print(data)


{'object_id': '570004012', 'query': 'dark blue polo shirt', 'label': 'GREAT'}
{'object_id': '806794003', 'query': 'dark blue polo shirt', 'label': 'GREAT'}
{'object_id': '751334002', 'query': 'dark blue polo shirt', 'label': 'BAD'}
{'object_id': '570004069', 'query': 'dark blue polo shirt', 'label': 'BAD'}
{'object_id': '570004026', 'query': 'dark blue polo shirt', 'label': 'OK'}
{'object_id': '570004078', 'query': 'dark blue polo shirt', 'label': 'BAD'}
{'object_id': '689107004', 'query': 'dark blue polo shirt', 'label': 'BAD'}
{'object_id': '806794004', 'query': 'dark blue polo shirt', 'label': 'BAD'}
{'object_id': '689113002', 'query': 'dark blue polo shirt', 'label': 'GREAT'}
{'object_id': '570004106', 'query': 'dark blue polo shirt', 'label': 'BAD'}


[Complete example](graded_input.jsonl)

These training labels are scoring results for a search on `dark blue polo shirt` as either `GREAT` or `BAD`. The Funetuning process will use this feedback to create a new Index optimized on the feedback. To download the full set of training labels and build a finetuned index with it:

In [ ]:
with open("graded_input.jsonl", "r") as file:
    training_data = [json.loads(line) for line in file]
    ft_index = index.finetune(training_data)

Executing this code will kick off the Finetuning process. You will receive an email when the finetuning process is completed.

## Compare results
Once you’re Finetuning is complete, you’ll be able to find your new index on the Indexes page in the [Console](https://app.objective.inc/dashboard/indexes). You’ll see the Finetuned tag on the new index.

Congratulations! 🎉 You’ve successfully learned how to optimize your existing search index using feedback data. This enables you to easily adapt the behavior of the system and optimize it to your use cases. The best part? Your index is now production-ready, so your users can immediately start enjoying the improved results.